# Get new FIRMS images from Google Earth Engine (GEE) API
Area of interest : Madagascar

Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap
from PIL import Image
from datetime import datetime, timedelta
import os

Authenticate to Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-kaloina4')  # 'ee-kaloina4' is the project name in GEE

Access FIRMS satellite images from 2000-11-01 to the current date focusing on Madagascar (cadence: 1 day)

In [3]:
firms_image = ee.ImageCollection("FIRMS")
pays = ee.FeatureCollection("FAO/GAUL/2015/level0");
madagascar = pays.filter(ee.Filter.eq('ADM0_NAME', 'Madagascar'));

def clip_to_madagascar(img):
  return img.clip(madagascar)

firms_images_mada = firms_image.map(clip_to_madagascar)
print("Number of FIRMS Madagascar images = " + str(firms_images_mada.size().getInfo()))

Number of FIRMS Madagascar images = 8944


## Visualization examples

In [4]:
exemple_mada_fire = firms_images_mada.filterDate("2024-01-13", "2024-05-28")
print("Number of FIRMS Madagascar images example = " + str(exemple_mada_fire.size().getInfo()))

Map=geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(madagascar, {'color': 'black'}, "Madagascar")
Map.centerObject(madagascar,5)
Map.addLayer(exemple_mada_fire,{},"ex:fire examples Mada")
Map

Number of FIRMS Madagascar images example = 136


Map(center=[-19.335922158747394, 46.7382233762819], controls=(WidgetControl(options=['position', 'transparent_â€¦

# Process FIRMS satellite images into grayscale PNG images
Available FIRMS satellite images: from 2000 - 11 - 01

We'll put our new data in Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Input : start and end date  

In [6]:
initial_date='2025-06-01'
until_date='2025-06-03'

We create folders for our new images

In [7]:
folder_path = "/content/drive/MyDrive/LIAM/Fire_Project/AI_project/data/raw/image_GEE/"
new_folder = folder_path + initial_date + ',' + until_date
os.makedirs(folder_path + initial_date + ',' + until_date, exist_ok=True)

In [8]:
start_date=initial_date

def add_days(date_str,dayadd=10):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    new_date = date_obj + timedelta(days=dayadd)
    return new_date.strftime("%Y-%m-%d")

gray_normalization = 180

while start_date < until_date:
    end_date=add_days(start_date)
    print('start_date: ' + start_date + ', end_date: '+ end_date)
    filter_mada_fire= firms_images_mada.filterDate(start_date, end_date)
    image=filter_mada_fire.max()
    imaget21=image.select('T21')
    arrayt21 = geemap.ee_to_numpy(imaget21, region=madagascar, scale=1000)
    arrayt21 = np.squeeze(arrayt21, axis=2)
    arrayt21[arrayt21 != 0] -= gray_normalization
    arrayt21 = arrayt21.astype(np.uint8)
    img = Image.fromarray(arrayt21, mode='L')
    directory=os.path.join(new_folder, start_date + ',' + end_date + '.png')
    img.save(directory)
    start_date=add_days(end_date,1)

print('end date: ' + end_date)

start_date: 2025-06-01, end_date: 2025-06-11
end date: 2025-06-11
